In [1]:
import torch
import torch.nn as nn
from TabNet import TabTransformer
import pandas as pd
from torch.utils.data import DataLoader,Dataset


In [2]:
trainDF=pd.read_csv("train.csv")
#testDF=pd.read_csv("test.csv")

In [3]:
from sklearn.model_selection import train_test_split
X=trainDF.values[:,:-1]
y=trainDF.values[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,stratify=y)

In [4]:
import numpy as np
np.max(X,axis=0)

array([ 4, 13,  4, 13,  4, 13,  4, 13,  4, 13], dtype=int64)

In [8]:
class CustomDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, idx):
        return self.x[idx,:], self.y[idx]   

batch_size=64
datasetTraining = CustomDataset(X_train,y_train)
datasetTest = CustomDataset(X_test,y_test)

train_dataloader = DataLoader(datasetTraining, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(datasetTest, batch_size=batch_size, shuffle=True)

In [12]:
model = TabTransformer(
    categories = (5, 14,  5, 14,  5, 14,  5, 14,  5, 14),      # tuple containing the number of unique values within each category
    num_continuous = 0,                # number of continuous values
    dim = 32,                           # dimension, paper set at 32
    dim_out = 1,                       
    depth = 6,                          # depth, paper recommended 6
    heads = 8,                          # heads, paper recommends 8
    attn_dropout = 0.1,                 # post-attention dropout
    ff_dropout = 0.1,                   # feed forward dropout
    mlp_hidden_mults = (32, 8),          # relative multiples of each hidden dimension of the last mlp to logits
    mlp_act = nn.Softmax(),                # activation for final mlp, defaults to relu, but could be anything else (selu etc)
)

[32, 1280, 320, 1]


In [13]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [15]:
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        labels=labels.reshape(-1,1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs,None)
        #print(outputs.size())
        #outputs = outputs.argmax()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

KeyboardInterrupt: 

In [88]:
from sklearn.metrics import classification_report
 
model.eval()
y_pred=[]
y_true=[]
for i, data in enumerate(test_dataloader, 0):
    inputs, labels = data
    outputs = model(inputs,None)
    y_pred.extend(outputs.argmax(dim=1).numpy())
    y_true.extend(labels)
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      4123
           1       0.99      1.00      1.00      3498
           2       0.85      0.90      0.88       398
           3       0.79      0.62      0.69       169
           4       0.80      0.13      0.22        31
           5       0.00      0.00      0.00        18
           6       0.33      0.17      0.22        12
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         2

    accuracy                           0.98      8254
   macro avg       0.48      0.38      0.40      8254
weighted avg       0.98      0.98      0.98      8254



C:\Users\MR.Mohebbian\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MR.Mohebbian\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MR.Mohebbian\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [17]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
 
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.89      0.82      4123
           1       0.70      0.67      0.68      3498
           2       0.33      0.05      0.09       398
           3       0.75      0.07      0.13       169
           4       0.33      0.03      0.06        31
           5       0.00      0.00      0.00        18
           6       0.00      0.00      0.00        12
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1

    accuracy                           0.73      8254
   macro avg       0.29      0.17      0.18      8254
weighted avg       0.71      0.73      0.71      8254



C:\Users\MR.Mohebbian\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MR.Mohebbian\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MR.Mohebbian\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  